<p style="text-align: justify">The parasitology research group of Prof. Estrada-Peña was interested in obtaining a phylogenetic view of the strains of the bacterium <em>Borrelia burgdorferi</em> s.l. (<strong><em>BB</em></strong>). It is a human pathogen transmitted by several species of ticks of the genus <em>Ixodes</em> and reservoired by more than 400 species of hosts. The interest to produce a phylogeny of the literally thousands of strains recorded in the northern hemisphere is to associate the genetic profile with the given species of vectors or reservoirs. The multilocus sequence typing (<strong><em>MLST</em></strong>) is a method that uses the sequence of 8 inhouse genes of BB that produces the best known approach to the typing of the strains of BB <a href="http://www.ncbi.nlm.nih.gov/pubmed/18574151" target="_blank">[1]</a>. The complete dataset is available at <a href="http://pubmlst.org/borrelia/" target="_blank">PubMLST</a>. The allel database contains 1535 sequences with 675 MLST profiles (on 15/Jul/2016).</p>

<p style="text-align: justify">We are going to use Biopython and MEvoLib to infer the phylogenetic tree. To do so, we need to compose the 675 MLST sequences from the 1535 alleles using the information provided by the MLST profile. The 8 allel files are available at our website in a <em>tar.gz</em> file (<a href="http://zaramit.unizar.es/download/datasets/borrelia_alleles.tar.gz">download</a>) and we have also created a Python file with the MLST profiles saved in one list (<a href="http://zaramit.unizar.es/download/datasets/mlst_info.py">download</a>). Therefore, the first step is to align each allel file with Mafft:</p>

In [ ]:
import os
from MEvoLib import Align

seq_files = [fname  for fname in os.listdir('.') if ( fname.endswith('.fas') )]
for fname in seq_files :
    Align.get_alignment('mafft', fname, 'fasta', args='default', outfile='{}.aln'.format(fname[:-4]),
                        outfile_format='fasta')

<p style="text-align: justify">Analyzing the resultant files we can see that only the <em>clpA</em>, <em>clpX</em> and <em>pyrG</em> alleles were modified (the other alleles were already aligned). The next step is to create a BioSeqs object containing all the aligned sequences to retrieve them really fast by their identifier. We can also save the resultant database to avoid the alignment process in the future:</p>

In [ ]:
import os
from MEvoLib.Fetch.BioSeqs import BioSeqs

aln_files = [fname  for fname in os.listdir('.') if ( fname.endswith('.aln') )]
seq_db = BioSeqs.from_seqfile(aln_files[0], 'fasta')
for fname in aln_files[1:] :
    seq_db.include(fname, 'fasta')
seq_db.write('bb_alleles.gb')

<p style="text-align: justify">Then, we are ready to generate the 675 MLST sequences from the profile information (stored in <em>mlst_info.MLST</em>). Every MLST sequence is composed by 8 allels (one from each file):</p>

In [ ]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from mlst_info import MLST

mlst_list = []
for entry in MLST :
    sequence = seq_db.data['clpA_{}'.format(entry[1])].seq + seq_db.data['clpX_{}'.format(entry[2])].seq + \
               seq_db.data['nifS_{}'.format(entry[3])].seq + seq_db.data['pepX_{}'.format(entry[4])].seq + \
               seq_db.data['pyrG_{}'.format(entry[5])].seq + seq_db.data['recG_{}'.format(entry[6])].seq + \
               seq_db.data['rplB_{}'.format(entry[7])].seq + seq_db.data['uvrA_{}'.format(entry[8])].seq
    record_id = 'mlst{:03d}'.format(entry[0])
    record = SeqRecord(sequence, id=record_id, name=record_id, description='')
    mlst_list.append(record)
num_seqs = SeqIO.write(mlst_list, 'mlst.fasta', 'fasta')
print(num_seqs)

<p style="text-align: justify">Finally, with all the MLST sequences saved in the <em>mlst.fasta</em> file, we can proceed to infer the phylogenetic tree for the BB. In this case, we are going to apply the GTR+CAT model through FastTree:</p>

In [ ]:
from MEvoLib import Inference

tree, score = Inference.get_phylogeny('fasttree', 'mlst.fasta', 'fasta', args='default',
                                      outfile='mlst.newick', outfile_format='newick')
print(score)